## W266 Final Project

In [1]:
import numpy as np
import pandas as pd
import re
import glob #this makes a list of all the files in a directory
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [3]:
files_list = glob.glob("./data/imsdb_scenes_dialogs_nov_2015/dialogs/*/*_dialog.txt")

file_array = np.array(files_list)
files = pd.DataFrame(file_array.reshape(-1,1), columns=["FilePath"])

title_list = []
genre_list = []
for i in files.index:
    genretitle = files['FilePath'][i][45:] #get rid of all the ./data/imsdb...
    a, b = genretitle.split("/")
    genre_list.append(a)
    title_list.append(b[:-11]) #take off '_dialog.txt' from the file name
    
    
files['Genre']=genre_list
files['Title']=title_list

files.head()


,FilePath,Genre,Title
0,./data/imsdb_scenes_dialogs_nov_2015/dialogs/H...,Horror,redridinghood
1,./data/imsdb_scenes_dialogs_nov_2015/dialogs/H...,Horror,aprilfoolsday
2,./data/imsdb_scenes_dialogs_nov_2015/dialogs/H...,Horror,hollowman
3,./data/imsdb_scenes_dialogs_nov_2015/dialogs/H...,Horror,queenofthedamned
4,./data/imsdb_scenes_dialogs_nov_2015/dialogs/H...,Horror,grudgethe


In [39]:
movie1 = pd.read_fwf('./data/imsdb_scenes_dialogs_nov_2015/dialogs/Adventure/avatar_dialog.txt',
                    header=None, names=["RawLine"],encoding='utf')
movie2 = pd.read_fwf('./data/imsdb_scenes_dialogs_nov_2015/dialogs/Adventure/1492conquestofparadise_dialog.txt',
                    header=None, names=["RawLine"],encoding='utf')
movie1.head(n=15)
### suggestion: always read in movies as UTF-8 so we don't have weird “&amp;amp;”  instead of &

,RawLine
0,AVATAR
1,Written by
2,James Cameron
3,VOICE (V.O.)
4,When I was lying there in the VA
5,"hospital, with a big hole blown through"
6,"the middle of my life, I started having"
7,these dreams of flying.
8,VOICE (V.O.)
9,"Sooner or later though, you always have"


In [40]:
def is_name(string):
    #string1 = re.sub('(V.O.)|CUT TO:','',string)
    ### nikki edit
    string1 = re.sub(r'\W+', '', re.sub('(V.O.)|CUT TO:','',string))
    n = re.match('[A-Z][A-Z]+', string1)
    if n: return True
    else: return False

def is_speech(string):
    if not is_name(string):
        return True
    else: return False

def is_cutto(string):
    n = re.match('CUT TO', string)
    #print n
    if n: return True
    else: return False
    

def parse_movie(movie, genre, title):   
#   Args:
#     movie is a pandas dataframe of the raw movie script input
#     genre is the movie genre from the folder it's read in from
#     title is the title (file name prior to _dialog.txt)
#   Output:
#     a dataframe of the form below
    #print genre, title
    curr_char = None
    line = 0
    title_column=[]
    genre_column=[]
    char_column=[]
    speech_column=[]
    line_column=[]
    pkey_column=[]
    for i in movie.index:
        title_column.append(title) 
        genre_column.append(genre) 
        current = str(movie['RawLine'][i])
        #current = str(current)
        #print current
        if current.lower() == 'nan':
            pass
            #print 'stop'

        elif is_cutto(current):
            char_column.append("")
            line_column.append(line)
            speech_column.append("")
            pkey_column.append("")

        elif is_name(current):

            #starting a new character. hold onto this name for the next lines
            #until you see another new character name
            #also try to remove things that are irrelevant to the name 
            ### nikki edit: add in str.strip() in case voice over left whitespaces
            curr_char = re.sub('\(V.O.\)|V.O.|\(O.S.\)|O.S.|\(CONT\'D\)|CONT\'D','',current).strip()
            char_column.append(curr_char)
            pkey_column.append((curr_char+'-'+title).lower().replace(' ', '_'))

            line += 1
            line_column.append(line)
            speech_column.append("")

        elif is_speech(current):

            char_column.append(curr_char)
            line_column.append(line)
            speech_column.append(current)
            if curr_char is not None:
                pkey_column.append((curr_char+'-'+title).lower())
                

    movie['Title']=title_column
    movie['Genre']=genre_column
    movie['Char']=char_column
    movie['LineID']=line_column
    movie['Speech']=speech_column
    movie['pkey']=pkey_column

    return movie



#parse_movie(movie1, 'Adventure', 'avatar')   
#parse_movie(movie2, 'Adventure', '1492conquestofparadise')
parse_movie(movie1, 'Adventure', 'avatar')
movie1.head(n=15)
    
#Intermediate dataframe should look something like this:

#RawLine                                  Title     Genre      Char   LineID     Speech

#JAKE (V.O.)                              Avatar    Adventure  JAKE   1 
#They can fix a spinal, if you've got the Avatar    Adventure  JAKE   1          They can fix... 
#money. But not on vet benefits, not in   Avatar    Adventure  JAKE   1          money. But not...


#And then hopefully we can eventually somehow aggregate by LineID to:

#Title   Genre      Char  Speech
#Avatar  Adventure  JAKE  They can fix a spinal, if you've got the money. But not on vet...


,RawLine,Title,Genre,Char,LineID,Speech,pkey
0,AVATAR,avatar,Adventure,AVATAR,1,,avatar-avatar
1,Written by,avatar,Adventure,AVATAR,1,Written by,avatar-avatar
2,James Cameron,avatar,Adventure,AVATAR,1,James Cameron,avatar-avatar
3,VOICE (V.O.),avatar,Adventure,VOICE,2,,voice-avatar
4,When I was lying there in the VA,avatar,Adventure,VOICE,2,When I was lying there in the VA,voice-avatar
5,"hospital, with a big hole blown through",avatar,Adventure,VOICE,2,"hospital, with a big hole blown through",voice-avatar
6,"the middle of my life, I started having",avatar,Adventure,VOICE,2,"the middle of my life, I started having",voice-avatar
7,these dreams of flying.,avatar,Adventure,VOICE,2,these dreams of flying.,voice-avatar
8,VOICE (V.O.),avatar,Adventure,VOICE,3,,voice-avatar
9,"Sooner or later though, you always have",avatar,Adventure,VOICE,3,"Sooner or later though, you always have",voice-avatar


In [8]:
movie1['Char'].value_counts()

JAKE                                       757
GRACE                                      434
QUARITCH                                   253
NEYTIRI                                    186
SELFRIDGE                                  137
NORM                                       123
(SUBTITLED)                                119
TRUDY                                       93
                                            87
MAX                                         69
MO'AT                                       63
TSU'TEY                                     55
WAINFLEET                                   33
EYTUKAN                                     18
PILOT                                       17
MED TECH                                    13
CREW CHIEF                                  13
TROOPER                                     11
JAKE (RECORDED)                             10
(QUIETLY)                                    8
VOICE                                        8
(ANGUISHED)  

In [7]:
movie1[movie1["Char"] == 'GRACE']

,RawLine,Title,Genre,Char,LineID,Speech
238,GRACE,avatar,Adventure,GRACE,61,
245,GRACE,avatar,Adventure,GRACE,64,
246,Norm. I hear good things about you.,avatar,Adventure,GRACE,64,Norm. I hear good things about you.
247,How's your Na'vi?,avatar,Adventure,GRACE,64,How's your Na'vi?
248,14.,avatar,Adventure,GRACE,64,14.
253,GRACE,avatar,Adventure,GRACE,66,
261,GRACE,avatar,Adventure,GRACE,71,
262,(turning to Jake),avatar,Adventure,GRACE,71,(turning to Jake)
263,"Yeah, yeah, I know who you are, and I",avatar,Adventure,GRACE,71,"Yeah, yeah, I know who you are, and I"
264,don't need you. I need your brother.,avatar,Adventure,GRACE,71,don't need you. I need your brother.


In [9]:
### keep only characters with at least 20 lines per movie

def top_chars(df, threshold = 20):
    #dframe = df
#     print dframe.head()
#     print type(dframe)
    threshold = threshold
    char_list = pd.DataFrame(df["Char"].value_counts())
    to_remove = char_list[char_list['Char'] <= threshold].index
    cl1 = char_list.drop(to_remove)
    cl1['line_count'] = cl1['Char']
    cl1['Char'] = cl1.index
    #char_list.reset_index(drop=True)
    query = 'SELECT d.*, c.line_count FROM df d INNER JOIN cl1 c on d.Char = c.Char'
    enriched_df = sqldf(query, locals())
    enriched_df = enriched_df[enriched_df['Char'] != '']
    #return cl1.head(20)
    return enriched_df

def pipeline(df,threshold=20):
    ### before top chars, we can add something about a proper title finder
    tc = top_chars(df, threshold)
    
    ### add something about the gender id
    ### add something about vocab enrichment
    ### output should be called final_df
    final_df = tc
    return tc
    

enriched_movie2 = pipeline(movie2)

In [11]:
enriched_movie2.head()

,RawLine,Title,Genre,Char,LineID,Speech,line_count
1,SANCHEZ,1492conquestofparadise,Adventure,SANCHEZ,2,,212
2,Your Majesty would wish to know the,1492conquestofparadise,Adventure,SANCHEZ,2,Your Majesty would wish to know the,212
3,true facts concerning the island of,1492conquestofparadise,Adventure,SANCHEZ,2,true facts concerning the island of,212
4,"Hispanola, our first settlement in",1492conquestofparadise,Adventure,SANCHEZ,2,"Hispanola, our first settlement in",212
5,"the New World, and the activities",1492conquestofparadise,Adventure,SANCHEZ,2,"the New World, and the activities",212


In [12]:
enriched_movie2['Char'].value_counts()

COLUMBUS         825
SANCHEZ          212
MARCHENA         122
AROJAZ           100
PINZON           100
FERNANDO          92
ISABEL            92
SANTANGEL         55
MOXICA            51
AL                44
UTAPAN            40
BEATRIX           38
MENDEZ            32
BOBADILLA         30
BARTOLOME         29
BROTHER BUYL      28
GIACOMO           24
SAILOR            22
ADMINISTRATOR     21
Name: Char, dtype: int64

In [13]:
enriched_movie1 = pipeline(movie1)

In [14]:
enriched_movie1.head()

,RawLine,Title,Genre,Char,LineID,Speech,line_count
1,JAKE (V.O.),avatar,Adventure,JAKE,4,,757
2,"They can fix a spinal, if you've got the",avatar,Adventure,JAKE,4,"They can fix a spinal, if you've got the",757
3,"money. But not on vet benefits, not in",avatar,Adventure,JAKE,4,"money. But not on vet benefits, not in",757
4,this economy.,avatar,Adventure,JAKE,4,this economy.,757
5,JAKE (V.O.),avatar,Adventure,JAKE,6,,757


In [15]:
enriched_movie1['Char'].value_counts()

JAKE           757
GRACE          434
QUARITCH       253
NEYTIRI        186
SELFRIDGE      137
NORM           123
(SUBTITLED)    119
TRUDY           93
MAX             69
MO'AT           63
TSU'TEY         55
WAINFLEET       33
Name: Char, dtype: int64

In [10]:
not_working=[]
char_list_to_label=[]

for i in files_list:
    title = i.split('/')[5].split('_')[0]
    genre = i.split('/')[4]
    if genre=='Western':
        print i
        #print genre
        title = title.strip()
        genre = genre.strip()
        movie = pd.read_fwf("%s" % i,header=None, names=["RawLine"],encoding='utf')
        #print movie
        try:
            parsed = parse_movie(movie, genre, title)
            #print parsed.head()
            final_df = pipeline(movie)
            char_list = final_df.drop_duplicates(subset='Char',keep='last')[['Char','Genre','Title','line_count']]
            char_list_to_label.append(char_list)
        except:
            not_working.append((genre,title))
        #print final_df.head()
        print "processing next movie"
chars= pd.concat(char_list_to_label)
#chars.to_csv('OutputWesternCharacters.csv', index=False)
print not_working
print chars

./data/imsdb_scenes_dialogs_nov_2015/dialogs/Western/djangounchained_dialog.txt
processing next movie
./data/imsdb_scenes_dialogs_nov_2015/dialogs/Western/danceswithwolves_dialog.txt
processing next movie
./data/imsdb_scenes_dialogs_nov_2015/dialogs/Western/bookofelithe_dialog.txt
processing next movie
./data/imsdb_scenes_dialogs_nov_2015/dialogs/Western/tombstone_dialog.txt
processing next movie
./data/imsdb_scenes_dialogs_nov_2015/dialogs/Western/mariachiel_dialog.txt
processing next movie
./data/imsdb_scenes_dialogs_nov_2015/dialogs/Western/baddayatblackrock_dialog.txt
processing next movie
./data/imsdb_scenes_dialogs_nov_2015/dialogs/Western/stationwest_dialog.txt
processing next movie
./data/imsdb_scenes_dialogs_nov_2015/dialogs/Western/wildwildwest_dialog.txt
processing next movie
./data/imsdb_scenes_dialogs_nov_2015/dialogs/Western/truegrit_dialog.txt
processing next movie
./data/imsdb_scenes_dialogs_nov_2015/dialogs/Western/wildbunchthe_dialog.txt
processing next movie
./data/i

In [70]:
avatar = parse_movie(movie1, 'Adventure', 'avatar') 
print avatar.head()

                            RawLine   Title      Genre    Char  LineID  \
0                            AVATAR  avatar  Adventure  AVATAR       1   
1                        Written by  avatar  Adventure  AVATAR       1   
2                     James Cameron  avatar  Adventure  AVATAR       1   
3                      VOICE (V.O.)  avatar  Adventure   VOICE       2   
4  When I was lying there in the VA  avatar  Adventure   VOICE       2   

                             Speech  
0                                    
1                        Written by  
2                     James Cameron  
3                                    
4  When I was lying there in the VA  


In [68]:
gender_labels = pd.read_csv('random_sampling_genderized.csv.csv',na_values="NA")
#print gender_labels.count()
#gender_labels = gender_labels[gender_labels['gender'] != 'NA']
#print gender_labels.count()
#gender_labels = gender_labels[gender_labels['gender'] is not None]
#print gender_labels.count()
gender_labels.head(n=40)

gender_labels = gender_labels.dropna(subset=['gender'])
gender_labels['gender'].value_counts()

M    758
F    392
Name: gender, dtype: int64

In [69]:
gender_labels.head()

,id,char,gender,genre,title,line_count,pkey
0,1,WILLY,M,Musical,sisteract,58,willy-sisteract
1,2,ATHOS,M,Action,manintheironmask,378,athos-manintheironmask
2,3,AUGUSTUS,M,Action,perfectcreature,222,augustus-perfectcreature
4,5,JIMMY,M,Romance,mulhollanddrive,43,jimmy-mulhollanddrive
5,6,DENNIS,M,Drama,backdraft,42,dennis-backdraft


In [74]:
all_movies_parsed = []
counter=0
for i in files_list:
    title = i.split('/')[5].split('_')[0]
    genre = i.split('/')[4]
    #print genre
    counter+=1
    print counter
    print i
    title = title.strip()
    genre = genre.strip()
    movie = pd.read_fwf("%s" % i,header=None, names=["RawLine"],encoding='utf')
    #print movie
    try:
        parsed = parse_movie(movie, genre, title)
        #print parsed.head()
        final_df = pipeline(movie)
        all_movies_parsed.append(final_df)
    except:
        pass
all_movies = pd.concat(all_movies_parsed)

print all_movies.head()

1
./data/imsdb_scenes_dialogs_nov_2015/dialogs/Crime/sisteract_dialog.txt
2
./data/imsdb_scenes_dialogs_nov_2015/dialogs/Comedy/sisteract_dialog.txt
3
./data/imsdb_scenes_dialogs_nov_2015/dialogs/Musical/sisteract_dialog.txt
                   RawLine      Title  Genre     Char  LineID  \
0                    VINCE  sisteract  Crime    VINCE      13   
1   (half to Christy, half  sisteract  Crime    VINCE      13   
2           to the mirror)  sisteract  Crime    VINCE      13   
3  You are something else.  sisteract  Crime    VINCE      13   
4                  CHRISTY  sisteract  Crime  CHRISTY      14   

                    Speech               pkey  line_count  
0                             vince-sisteract         231  
1   (half to Christy, half    vince-sisteract         231  
2           to the mirror)    vince-sisteract         231  
3  You are something else.    vince-sisteract         231  
4                           christy-sisteract        1234  


,RawLine,Title,Genre,Char,LineID,Speech,pkey,line_count
1,JAKE (V.O.),avatar,Action,JAKE,4,,jake-avatar,757
2,"They can fix a spinal, if you've got the",avatar,Action,JAKE,4,"They can fix a spinal, if you've got the",jake-avatar,757
3,"money. But not on vet benefits, not in",avatar,Action,JAKE,4,"money. But not on vet benefits, not in",jake-avatar,757
4,this economy.,avatar,Action,JAKE,4,this economy.,jake-avatar,757
5,JAKE (V.O.),avatar,Action,JAKE,6,,jake-avatar,757


In [90]:
query = 'SELECT m.*, g.gender FROM all_movies m INNER JOIN gender_labels g on m.pkey = g.pkey'
with_gender = sqldf(query, locals())
with_gender = with_gender[with_gender['Char'] != '']
with_gender = with_gender[with_gender['Speech'] != '']
with_gender = with_gender.drop_duplicates(subset=['pkey','LineID','Speech'],keep='last')
with_gender = with_gender.groupby(['gender','Char','Title','Genre','pkey','LineID']).Speech.unique().reset_index()            

In [91]:
with_gender

,gender,Char,Title,Genre,pkey,LineID,Speech
0,M,WILLY,sisteract,Musical,willy-sisteract,120,"[I mean, won't she talk? She seemed kind, of u..."
1,M,WILLY,sisteract,Musical,willy-sisteract,125,[Christy?]
2,M,WILLY,sisteract,Musical,willy-sisteract,412,"[It's that call you were expecting,, Vince.]"
3,M,WILLY,sisteract,Musical,willy-sisteract,579,[What? Christy?]
4,M,WILLY,sisteract,Musical,willy-sisteract,582,[Gimme a hand here.]
5,M,WILLY,sisteract,Musical,willy-sisteract,946,"[Yeah, right -- when I see it.]"
6,M,WILLY,sisteract,Musical,willy-sisteract,970,[Go!]
7,M,WILLY,sisteract,Musical,willy-sisteract,979,"[Forget her! We got the one we need!, Step on..."
8,M,WILLY,sisteract,Musical,willy-sisteract,1031,"[She's a nun, Vince.]"
9,M,WILLY,sisteract,Musical,willy-sisteract,1037,"[She's acting weird, Vince. Ever since, we pi..."
